In [1]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import LogNorm
import datetime
import os
dp = '/home/ebert/jupyter-notebooks/FNode-data'
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]

exclude = [2]

In [2]:
exp_names = [
    '2018_07_11_09_29_44_trap-lifetime',
    '2018_07_11_13_45_13_trap-lifetime-2'
]

single_only = True

rets = 'retention'
errs = 'error'
if single_only:
    rets = 'conditional_retention'
    errs = 'conditional_retention_err'

q = qdp.QDP(base_data_path=dp)
for exp_name in exp_names:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
    q.load_data_file(experiment_file)

print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[33],[3]]
rb_cuts = [[31],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=exclude, ncondition=1)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"

q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    xlab = 'iterator'

tbl_str = "ivar:\t{0:.3f}\tloading:{1:.3f}\tretention:{2:.3f} +- {3:.3f}"
for i in range(len(retention[rets].flatten())):
    print(tbl_str.format(
        retention['ivar'].flatten()[i],
        retention['loading'].flatten()[i],
        retention[rets].flatten()[i],
        retention[errs].flatten()[i]
    ))

data at: 2018_07_11/2018_07_11_09_29_44_trap-lifetime/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
data at: 2018_07_11/2018_07_11_13_45_13_trap-lifetime-2/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'gap_time']
[    79.     91.  10000.]
[   82.    100.   4005.1]
loading
loading
loading
[ 65.  63.   5.]
[  96.  100.    0.]
loading
loading
no loading observed
[    65.     52.  10000.]
[ 106.  100.    0.]
loading
loading
no loading observed
[ 62.  51.   5.]
[ 137.  100.    0.]
loading
loading
no loading observed
[    90.     45.  10000.]
[ 192.  102.    0.]
loading
loading
no loading observed
[ 56.  43.   5.]
[ 153.  103.    0.]
loading
loading
no loading observed
[    34.     31.  10000.]
[ 147.  101.    0.]
loading
loading
no loading observed
[ 16.  21.   5.]
[ 79.  98.   0.]
loading
loading
no loading observed
[     0.     29.  10000.]
[  2.  99.   0.]
loading
loading
no loading observed
[ 16.  23.   5.]
[ 

/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:133: RuntimeWarning: invalid value encountered in sqrt
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))


In [3]:
q.save_experiment_data(filename_prefix='lifetime')
legends = ['{}'.format(x) for x in [1,2]]

In [86]:
fig, ax = plt.subplots()
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
roi = 0
# fits = []
ivar = np.concatenate(np.linspace(8.5,1e4,6)
retention['ivar'][1,:,0] = np.linspace(1.2e4,2e4,5)
for e in range(len(retention['ivar'])):
#     fits.append(np.polyfit(retention['ivar'][e,:,roi], retention['retention'][e,:,roi], 1))
    ax.errorbar(retention['ivar'][e,:,0], retention['retention'][e,:,roi],
                yerr=retention['error'][e,:,roi],
                fmt = colors[e]+'o-',
                label=legends[e]
    )
        
#     ax.plot(-8.6-np.linspace(0,2,6), retention['loading'][e,:,roi], colors[e]+'x')
#     ax.plot(retention['ivar'][e,:,roi], np.poly1d(fits[-1])(retention['ivar'][e,:,roi]), colors[e]+'-' ,label=legends[e])
ax.legend(bbox_to_anchor=(0.85, 0.95), loc=1, borderaxespad=0.)
ax.set_ylim(0., 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=2')
fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'cs-uwave-ramsey.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

ValueError: could not broadcast input array from shape (5) into shape (6)

In [3]:
import scipy.optimize as optimize

fig, ax = plt.subplots()
ret = []
err = []
fmt_map = ['bo', 'rd']
ivar = [
    np.delete(np.concatenate((np.linspace(8.5,1e4,6), np.linspace(1.2e4,2e4,5)))/1e3, -3),
    np.concatenate((np.linspace(8.5,1e4,6), np.linspace(1.2e4,2e4,5)))/1e3
]

def expon(x, a, tau, b):
    return b + a*np.exp(-x/tau)

for r in range(2):
    ret.append(np.concatenate((retention[rets][0,:,r], retention[rets][1,:-1,r])))
    err.append(np.concatenate((retention[errs][0,:,r], retention[errs][1,:-1,r])))
    if r == 0:
        ret[r] = np.delete(ret[r], -3)
        err[r] = np.delete(err[r], -3)
    guess = [1,10]
    popt, pcov = optimize.curve_fit(
        lambda t, a, tau: expon(t, a, tau, 0),
        ivar[r],
        ret[r],
        p0=guess,
        sigma=err[r]
    )
    print(popt)
    print(np.sqrt(np.diagonal(np.abs(pcov))))
    xs = np.linspace(0,25,100)
    ax.plot(xs, expon(xs, popt[0], popt[1], 0), fmt_map[r][0]+'--', alpha=0.6)
    ax.errorbar(ivar[r], ret[r], yerr=err[r], **marker_styles[r])
        
ax.set_ylim(0., 1.0)
ax.set_xlabel('Time Between Readouts (s)')
ax.set_ylabel('Retention')
fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'trap-lifetime_20180711.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[  0.95692399  11.29052678]
[ 0.01630036  0.48585541]
[  0.91393715  13.4564238 ]
[ 0.02352775  0.83475513]


In [4]:
retention['error'][0,:,0]

array([ 0.02406789,  0.04060384,  0.04201635,  0.04204159,  0.04134656,
        0.04086486])

In [6]:
retention[errs][1,:,0]

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [45]:
fig, ax = plt.subplots()
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
roi = 1
# fits = []
for e in range(len(retention['ivar'])):
#     fits.append(np.polyfit(retention['ivar'][e,:,roi], retention['retention'][e,:,roi], 1))
    l = len(retention['ivar'][e,:,roi])
#     if e == 3:
#         l = 4
    ax.errorbar(retention['ivar'][e,:l,roi], retention['retention'][e,:l,roi],
                yerr=retention['error'][e,:l,roi],
                fmt = colors[e]+'o-',
                label=legends[e]
    )
        
#     ax.plot(retention['ivar'][e,:,roi], retention['loading'][e,:,roi], colors[e]+'x')
#     ax.plot(retention['ivar'][e,:,roi], np.poly1d(fits[-1])(retention['ivar'][e,:,roi]), colors[e]+'-' ,label=legends[e])
ax.legend(bbox_to_anchor=(0.85, 0.95), loc=1, borderaxespad=0.)
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('Retention')
fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'rb_retention.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [6]:
import scipy.optimize as optimize

def gaussian(x, a, x0, s, b):
    return b + a*np.exp(-0.5*((x-x0)/s)**2)

def rfe(x, a, f, phi, b, tau):
    return b + a*np.cos(2*np.pi*f*x+phi)*np.exp(-x/tau)

In [82]:
fig, ax = plt.subplots()
xdat = []
ydat = []
edat = []
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
roi = 1
# fits = []
for e in range(len(retention['ivar'])):
#     fits.append(np.polyfit(retention['ivar'][e,:,roi], retention['retention'][e,:,roi], 1))
    xpts = [[ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ][:13],[ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ,
        2.2,  2.4,  2.6,  2.8,  3. ][:13]][e]
    ax.errorbar(xpts, retention['retention'][e,:,roi],
                yerr=retention['error'][e,:,roi],
                fmt = colors[e]+'o',
                label=legends[e]
    )
    xdat += xpts
    ydat += retention['retention'][e,:,roi].tolist()
    edat += retention['error'][e,:,roi].tolist()
# print(xdat)
# print(ydat)
# print(edat)
ax.legend(bbox_to_anchor=(0.85, 0.95), loc=1, borderaxespad=0.)
ax.set_ylim(0., 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=2')

guess = [ 0.7,  1.2, 3, 0.5, 3]
popt, pcov = optimize.curve_fit(
    rfe,
    xdat,
    ydat,
    p0=guess,
    sigma=edat
)
xs = np.linspace(min(xdat),max(xdat),100)
ax.plot(xs, rfe(xs, *popt))

fit_data = "a: {:.3f}\nf: {:.5f}\nphi: {:.3f}\nb: {:.3f}\ntau: {:.3f}"
ax.text(4.5, 0.1, fit_data.format(*popt), fontsize=12)

fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'rb-uwave-ramsey.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

In [40]:
xdat

[[-1.0, -1.0],
 [-0.8, -0.8],
 [-0.6, -0.6],
 [-0.3999999999999999, -0.3999999999999999],
 [-0.19999999999999996, -0.19999999999999996],
 [0.0, 0.0],
 [0.20000000000000018, 0.20000000000000018],
 [0.40000000000000013, 0.40000000000000013],
 [0.6000000000000001, 0.6000000000000001],
 [0.8, 0.8],
 [1.0, 1.0],
 [-0.5, -0.5],
 [-0.7, -0.7],
 [-0.9, -0.9],
 [-1.1, -1.1],
 [-1.3, -1.3],
 [-1.5, -1.5],
 [-1.7000000000000002, -1.7000000000000002],
 [-1.9000000000000001, -1.9000000000000001],
 [-2.1, -2.1],
 [-2.3, -2.3],
 [-2.5, -2.5],
 [-2.0, -2.0],
 [-2.1, -2.1],
 [-2.2, -2.2],
 [-2.3, -2.3],
 [-2.4, -2.4],
 [-2.5, -2.5],
 [-2.6, -2.6],
 [-2.7, -2.7],
 [-2.8, -2.8],
 [-2.9, -2.9],
 [-3.0, -3.0],
 [-3.5, -3.5],
 [-3.3333333333333335, -3.3333333333333335],
 [-3.1666666666666665, -3.1666666666666665],
 [-3.0, -3.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.35807730942e-312, 1.094299833e-315],
 [0.0, 0.0],
 [0.0, nan],
 [0.0, 0.0]]

In [13]:
fig, ax = plt.subplots()
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
roi = 0
# fits = []
for e in range(len(retention['ivar'])):
#     fits.append(np.polyfit(retention['ivar'][e,:,roi], retention['retention'][e,:,roi], 1))
    l = len(retention['ivar'][e,:,roi])
#     if e == 3:
#         l = 4
    ax.plot([1,0], retention['loading'][e,:l,roi],
            colors[e]+'o-',
            label=legends[e]
    )
        
#     ax.plot(retention['ivar'][e,:,roi], retention['loading'][e,:,roi], colors[e]+'x')
#     ax.plot(retention['ivar'][e,:,roi], np.poly1d(fits[-1])(retention['ivar'][e,:,roi]), colors[e]+'-' ,label=legends[e])
ax.legend(bbox_to_anchor=(0.85, 0.95), loc=1, borderaxespad=0.)
ax.set_ylim(0., .3)
ax.set_xlabel("Rb loading")
ax.set_ylabel('Loading')
fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'cs_rb-collision_loading.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

In [9]:
fig, ax = plt.subplots()
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
roi = 0
# fits = []
for e in range(len(retention['ivar'])):
#     fits.append(np.polyfit(retention['ivar'][e,:,roi], retention['retention'][e,:,roi], 1))
    l = len(retention['ivar'][e,:,roi])
#     if e == 3:
#         l = 4
    ax.plot(retention['loading'][e,:l,1], retention['loading'][e,:l,roi],
            colors[e]+'o-',
            label=legends[e]
    )
        
#     ax.plot(retention['ivar'][e,:,roi], retention['loading'][e,:,roi], colors[e]+'x')
#     ax.plot(retention['ivar'][e,:,roi], np.poly1d(fits[-1])(retention['ivar'][e,:,roi]), colors[e]+'-' ,label=legends[e])
ax.legend(bbox_to_anchor=(0.85, 0.95), loc=1, borderaxespad=0.)
ax.set_ylim(0., .3)
ax.set_xlabel("Rb loading")
ax.set_ylabel('Cs Loading')
fmts = ['pdf', 'png']
fn = os.path.join(dp, os.path.dirname(experiment_file), 'cs_rb-collision_loadingXY.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()